<a href="https://colab.research.google.com/github/KamonohashiPerry/PyTorch/blob/master/PyTorch_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorchのインストール
+ Colabの編集→ノートブックの設定でGPUモードにすることをお忘れなく。

In [1]:
# pip install http://download.pytorch.org/whl/cu80/torch-0.4.0-cp36-cp36m-linux_x86_64.whl

     |████████████████████████████████| 484.0MB 89.0MB/s 
ERROR: torchvision 0.4.2 has requirement torch==1.3.1, but you'll have torch 0.4.0 which is incompatible.
ERROR: fastai 1.0.59 has requirement torch>=1.0.0, but you'll have torch 0.4.0 which is incompatible.
  Found existing installation: torch 1.3.1
    Uninstalling torch-1.3.1:
      Successfully uninstalled torch-1.3.1


In [2]:
# pip install torchvision

     |████████████████████████████████| 734.6MB 23kB/s 
  Found existing installation: torch 0.4.0
    Uninstalling torch-0.4.0:
      Successfully uninstalled torch-0.4.0


In [3]:
# pip install tqdm

インストールできたかの確認

In [0]:
import torch

In [0]:
torch.tensor([1, 2, 3]).to("cuda:0")

In [0]:
# !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
# !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
# !apt-get update -qq 2>&1 > /dev/null
# !apt-get -y install -qq google-drive-ocamlfuse fuse

# from google.colab import auth
# auth.authenticate_user()
# from oauth2client.client import GoogleCredentials
# creds = GoogleCredentials.get_application_default()
# import getpass
# !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL

# vcode = getpass.getpass()
# !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
# # drive mean root directory of  google drive
# !mkdir -p drive
# !google-drive-ocamlfuse drive
# !ls drive/"Colab Notebooks"/PyTorch

In [0]:
import torch
from torch import nn, optim
from torch.utils.data import (Dataset, DataLoader, TensorDataset)
import tqdm

import warnings
warnings.filterwarnings('ignore')

In [10]:
!wget http://files.grouplens.org/datasets/movielens/ml-20m.zip

--2019-12-22 15:07:18--  http://files.grouplens.org/datasets/movielens/ml-20m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 198702078 (189M) [application/zip]
Saving to: ‘ml-20m.zip’

ml-20m.zip          100%[===================>] 189.50M  26.1MB/s    in 7.9s    

2019-12-22 15:07:26 (23.9 MB/s) - ‘ml-20m.zip’ saved [198702078/198702078]



In [11]:
!unzip ml-20m.zip

Archive:  ml-20m.zip
   creating: ml-20m/
  inflating: ml-20m/genome-scores.csv  
  inflating: ml-20m/genome-tags.csv  
  inflating: ml-20m/links.csv        
  inflating: ml-20m/movies.csv       
  inflating: ml-20m/ratings.csv      
  inflating: ml-20m/README.txt       
  inflating: ml-20m/tags.csv         


In [12]:
pwd

'/content'

In [13]:
ls

ml-20m/  ml-20m.zip  sample_data/


In [22]:
import pandas as pd

# 訓練データとテストデータを分けるのに使用する
from sklearn import model_selection

df = pd.read_csv("/content/ml-20m/ratings.csv")

# Xは(userID, movieId)のペア
X = df[["userId", "movieId"]].values
X

array([[     1,      2],
       [     1,     29],
       [     1,     32],
       ...,
       [138493,  69644],
       [138493,  70286],
       [138493,  71619]])

In [23]:
Y = df[["rating"]].values
Y

array([[3.5],
       [3.5],
       [3.5],
       ...,
       [3. ],
       [5. ],
       [2.5]])

In [0]:
# 訓練データとテストデータを9:1に分割
train_X, test_X, train_Y, test_Y = model_selection.train_test_split(X, Y, test_size=0.1)

In [25]:
train_X.shape

(18000236, 2)

In [26]:
test_X.shape

(2000027, 2)

In [27]:
train_Y.shape

(18000236, 1)

In [28]:
test_Y.shape

(2000027, 1)

In [0]:
# XはIDで整数なのでint64、Yは実数値なのでfloat32のTensorに変換する
train_dataset = TensorDataset(torch.tensor(train_X, dtype=torch.int64),
                               torch.tensor(train_Y, dtype=torch.float32))

In [36]:
train_dataset.tensors

(tensor([[ 83629,    256],
         [120667,   4367],
         [ 52877,  59126],
         ...,
         [ 85951,    318],
         [ 16753,   2787],
         [ 62738,    942]]), tensor([[3.],
         [4.],
         [4.],
         ...,
         [4.],
         [2.],
         [5.]]))

In [0]:
test_dataset = TensorDataset(torch.tensor(test_X, dtype=torch.int64),
                               torch.tensor(test_Y, dtype=torch.float32))

In [38]:
test_dataset.tensors

(tensor([[130767,  50147],
         [131499,    105],
         [ 66042,    356],
         ...,
         [ 11583,   6711],
         [ 74626,   1304],
         [ 70313,   1197]]), tensor([[1.0000],
         [4.0000],
         [5.0000],
         ...,
         [2.5000],
         [4.0000],
         [5.0000]]))

In [0]:
train_loader = DataLoader(train_dataset, batch_size=1024, num_workers=4,shuffle=True)

In [47]:
print(train_loader.batch_size)
print(train_loader.num_workers)
print(train_loader.dataset)

1024
4


In [50]:
train_loader.dataset.tensors

(tensor([[ 83629,    256],
         [120667,   4367],
         [ 52877,  59126],
         ...,
         [ 85951,    318],
         [ 16753,   2787],
         [ 62738,    942]]), tensor([[3.],
         [4.],
         [4.],
         ...,
         [4.],
         [2.],
         [5.]]))

In [0]:
test_loader = DataLoader(test_dataset, batch_size=1024,num_workers=4)